In [ ]:
import io

import cv2
import ipywidgets
import numpy as np
from ipywebrtc import CameraStream, ImageRecorder

In [ ]:
camera = CameraStream(
    constraints={
        'facing_mode': 'user',
        'audio': False,
        'video': { 'width': 450 }
    }
)

image_recorder = ImageRecorder(stream=camera)

In [ ]:
# download file at https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
def process(img_bytes):
    arr = np.asarray(image_recorder.image.value, np.uint8)
    img_np = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
    
    # Face detection
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Process only one face
    if len(faces) != 1:
        return None

    # Pad rectangle
    pad_xy = 40, 70
    rect = faces[0]
    rect[0] -= pad_xy[0]
    rect[1] -= pad_xy[1]
    rect[2] += pad_xy[0]
    rect[3] += pad_xy[1]

    # two passes grabcut for background delineation
    mask = np.zeros(img_np.shape[:2], dtype=np.uint8)
    mask2 = np.zeros(img_np.shape[:2], dtype=np.uint8)
    bgd_model = np.zeros((1, 65), np.float64)
    fgd_model = np.zeros((1, 65), np.float64)
    cv2.grabCut(img_np, mask, rect, bgd_model, fgd_model, 1, cv2.GC_INIT_WITH_RECT)
    cv2.grabCut(img_np, mask, None, bgd_model, fgd_model, 1, cv2.GC_INIT_WITH_MASK)

    mask = np.where((mask == cv2.GC_PR_BGD) | (mask == cv2.GC_BGD), 0, 1).astype('uint8')
    masked = gray * mask
    
    # Edge detection
    sobelxy = cv2.Sobel(src=masked, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5)
    
    # Blend
    result_arr = np.maximum(masked * 0.5 + sobelxy, 1) 
    
    _, result = cv2.imencode('.png', result_arr)
    return result.tobytes()

In [ ]:
out = ipywidgets.Image()
stop_process = False

def process_image(_):
    if stop_process:
        return

    # update only if processing is successful
    result = process(image_recorder.image.value)
    if result is not None:
        out.value = result

    # infinite process loop (force toggle recording)
    image_recorder.recording = False
    image_recorder.recording = True

image_recorder.image.observe(process_image, names=['value'])

ipywidgets.HBox([image_recorder.stream, out])

In [ ]:
image_recorder.recording = True

In [ ]:
stop_process = True

In [ ]:
ipywidgets.Widget.close_all()